In [1]:
import json
from base import BaseModel 
from lite_llm import LiteLLMInput
from lite_llm import LiteLLMSetting
from lite_llm import LiteLLMService
from lite_llm import Role 
from lite_llm import CompletionMessage
from pydantic import Field
import base64
import json
from pydantic import HttpUrl, SecretStr
from generation.shared.utils import get_previous_lectures
from storage.minio import MinioInput, MinioService, MinioSetting

minio_setting = MinioSetting(
    endpoint="localhost:9000",
    access_key="minioadmin",
    secret_key="minioadmin123",
    secure=False,
)
        
minio_service = MinioService(settings=minio_setting)

litellm_setting = LiteLLMSetting(
    url=HttpUrl("http://localhost:9510"),
    token=SecretStr("abc123"),
    model="gemini-2.5-flash",
    frequency_penalty=0.0,
    n=1,
    temperature=0.0,
    top_p=1.0,
    max_completion_tokens=10000,
    dimension=1536,
    embedding_model="gemini-embedding"
)

litellm_service = LiteLLMService(litellm_setting=litellm_setting)

class Question(BaseModel):
    question: str = Field(..., description="The quiz question")
    answer: str = Field(..., description="The correct answer to the quiz question")
    distractors: list[str] = Field(..., description="A list of incorrect answers to the quiz question")
    explanation: str = Field(..., description="An explanation of the correct answer")
    
class Questions(BaseModel):
    questions: list[Question] = Field(..., description="A list of quiz questions")

system_prompt = """
You are an expert educational content creator and quiz designer. Your task is to create comprehensive and challenging quiz questions based on the provided PDF lecture material and summaries from previous lectures.

Guidelines for creating quiz questions:

1. **Content Coverage**: 
   - Focus primarily on the current PDF lecture content (80% of questions)
   - Include some questions that connect current content with previous lectures (20% of questions)
   - Cover key concepts, definitions, algorithms, formulas, and practical applications

2. **Question Types and Difficulty**:
   - Create questions of varying difficulty levels (30% easy, 50% medium, 20% hard)
   - Include conceptual understanding questions, not just memorization
   - Add application-based questions that test practical knowledge
   - Include questions about advantages/disadvantages of different approaches

3. **Question Structure**:
   - Each question should have exactly 4 answer choices (1 correct + 3 distractors)
   - Distractors should be plausible but clearly incorrect to knowledgeable students
   - Avoid "all of the above" or "none of the above" options
   - Make questions clear and unambiguous

4. **Answer Explanations**:
   - Provide detailed explanations for why the correct answer is right
   - Briefly explain why the distractors are incorrect when helpful
   - Include relevant formulas, concepts, or references to lecture content

5. **Content Integration**:
   - When using previous lecture summaries, create questions that show progression of learning
   - Highlight connections between current and previous topics
   - Ensure questions test understanding rather than rote memorization

6. **Quality Standards**:
   - Questions should be appropriate for university-level students
   - Use precise technical terminology from the field
   - Ensure mathematical notation and formulas are accurate
   - Create questions that encourage critical thinking

Generate 8 high-quality quiz questions following these guidelines.
"""

user_prompt = """
Bạn là một chuyên gia thiết kế đề thi và nội dung giáo dục với nhiều năm kinh nghiệm trong việc tạo ra các bài kiểm tra chất lượng cao cho sinh viên đại học.

NHIỆM VỤ: Tạo 8 câu hỏi trắc nghiệm bằng tiếng Việt dựa trên các chủ đề được cung cấp dưới đây.

CÁC CHỦ ĐỀ:
{topics}

HƯỚNG DẪN CHI TIẾT:

1. **Cấu trúc câu hỏi**:
   - Mỗi câu hỏi phải có đúng 4 lựa chọn (A, B, C, D)
   - 1 đáp án đúng và 3 đáp án nhiễu hợp lý
   - Các đáp án nhiễu phải có tính thuyết phục cao, không quá dễ loại bỏ

2. **Phân bổ độ khó**:
   - 30% câu hỏi dễ (kiến thức cơ bản, định nghĩa)
   - 50% câu hỏi trung bình (ứng dụng khái niệm, phân tích)
   - 20% câu hỏi khó (tổng hợp, đánh giá, giải quyết vấn đề)

3. **Yêu cầu nội dung**:
   - Tạo câu hỏi theo đúng thứ tự các chủ đề được cung cấp
   - Tập trung vào hiểu biết sâu về khái niệm, không chỉ ghi nhớ
   - Bao gồm các câu hỏi về ứng dụng thực tế và tình huống cụ thể
   - Sử dụng thuật ngữ chuyên môn chính xác

4. **Tiêu chuẩn chất lượng**:
   - Câu hỏi phải rõ ràng, không gây nhầm lẫn
   - Tránh sử dụng "tất cả các đáp án trên" hoặc "không có đáp án nào đúng"
   - Đảm bảo tính khách quan và công bằng
   - Phù hợp với trình độ sinh viên đại học

5. **Giải thích đáp án**:
   - Cung cấp lý do chi tiết tại sao đáp án được chọn là đúng
   - Giải thích ngắn gọn tại sao các đáp án khác không chính xác (khi cần thiết)
   - Tham chiếu đến khái niệm, công thức hoặc nguyên lý liên quan

Hãy tạo ra 8 câu hỏi chất lượng cao thể hiện sự chuyên nghiệp và am hiểu sâu về lĩnh vực giáo dục."""

pdf_path = "/home/lehoangvu/KLTN/lectures/int3405/tuan-1/Lecture03_Linear Regression.pdf"

# summaries = get_previous_lectures(minio_service, "rl2025", 6)

# with open(pdf_path, 'rb') as pdf_file:
#     pdf_bytes = pdf_file.read()
course_code = "dsa2025"
for week_number in range(1, 9):
   with open(f'/home/lehoangvu/KLTN/MCQs_4o_mini/{course_code}/week{week_number}_pipeline.json') as f:
      pipeline_output = json.load(f)
      
   topics = [item['topic'] for item in pipeline_output['questions']]
   output = await litellm_service.process_async(
      inputs=LiteLLMInput(
         messages=[
               # CompletionMessage(
               #     role=Role.SYSTEM,
               #     content=system_prompt
               # ),
               # CompletionMessage(
               #     role=Role.USER,
               #     file_url=f"data:application/pdf;base64,{base64.b64encode(pdf_bytes).decode('utf-8')}"
               # ),
               CompletionMessage(
                  role=Role.USER,
                  content=user_prompt.format(topics=json.dumps(topics, ensure_ascii=False, indent=2))
               )
         ],
         response_format=Questions,
         model="gpt-4o-mini",
         temperature=0.0,
         max_completion_tokens=15000,
         top_p=1.0,
         frequency_penalty=0.0,
         n=1,
      )
   )
   with open(f"MCQs_4o_mini/{course_code}/week{week_number}_baseline.json", "w", encoding='utf-8') as f:
      json.dump(
         output.response.model_dump(),
         f,
         indent=4,
         ensure_ascii=False
    )